In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, r2_score

sns.set(rc={'figure.figsize':(20,20)})

current_dir = os.path.abspath('')
parent_dir = os.path.dirname(current_dir)

In [2]:
regions_data = pd.read_csv(os.path.join(parent_dir, 'datasets', 'region-economics-prediction', 'train.csv'))
regions_data.loc[32, 'x59'] = 0.0
regions_data.loc[20, 'x4'] = 4379
regions_data['x4'] = regions_data['x4'].astype('float64')
regions_data['x59'] = regions_data['x59'].astype('float64')
regions_data['x58'] = regions_data['x58'].astype('float64')

#regions_data.info()

# проверка на NaN-ы
for i in regions_data.columns:
    if regions_data[i].isnull().values.any():
        print(i)
        regions_data[i].fillna(regions_data[i].mean(), axis=0, inplace=True)

regions_data

x38
x59
x60


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x57,x58,x59,x60,x61,x62,x63,x64,Y1,Y2
0,32.0,60.0,15.0,1871.0,56.0,6.0,10.0,0.0,58653.95,176.4,...,175.0,311667.0,154.27,18.48,27338.4,99.9,74199.0,1002.69,0,1252.25
1,14.0,7.0,2.0,585.0,2.0,2.0,4.0,2.0,34613.83,848.0,...,559.0,1497992.0,109.64,1.54,17209.5,112.9,-7194.0,-97.22,0,418.14
2,42.0,131.0,32.0,3369.0,102.0,15.0,38.0,15.0,36801.25,719.6,...,579.5,1207875.0,251.65,7.39,15966.3,99.3,19263.0,260.31,0,480.15
3,48.0,89.0,14.0,2630.0,183.0,20.0,1.0,1.0,37973.60,732.0,...,500.2,1098257.0,229.92,10.30,17529.3,94.9,64773.0,875.31,0,544.36
4,14.0,49.0,3.0,698.0,55.0,2.0,9.0,3.0,33297.72,149.8,...,122.7,269984.0,87.40,1.61,17571.8,96.7,2268.0,30.65,0,448.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,135.0,154.0,33.0,6216.0,447.0,48.0,114.0,10.0,49722.15,815.6,...,515.7,1082662.0,102.36,9.80,20201.0,126.9,28204.0,381.14,0,771.30
64,72.0,234.0,24.0,5516.0,161.0,34.0,110.0,29.0,58046.47,904.4,...,893.1,1687654.0,545.90,23.07,19799.1,110.6,1265422.0,17100.30,1,1151.50
65,132.0,376.0,81.0,23664.0,676.0,125.0,159.0,35.0,38877.22,1619.8,...,1349.0,2785836.0,301.91,37.53,17726.7,111.9,175754.0,2375.05,0,627.63
66,90.0,140.0,16.0,9047.0,387.0,30.0,77.0,15.0,38654.35,800.4,...,619.9,1245619.0,264.48,15.32,17109.5,100.0,30067.0,406.31,0,536.38


In [3]:
regions_final_test_data = pd.read_csv(os.path.join(parent_dir, 'datasets', 'region-economics-prediction', 'test.csv'))
regions_final_test_data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64
0,132.0,473.0,57.0,12201,461.0,78.0,85.0,17.0,40450.05,1032.4,...,-310.0,9.15,960.6,1877844.0,120.69,12.55,20830.5,75.0,225557.0,3048.07
1,51.0,125.0,26.0,4734,166.0,22.0,77.0,1.0,39120.38,998.2,...,6436.0,7.72,794.5,1541259.0,175.28,13.76,16022.7,108.6,935400.0,12640.54
2,236.0,672.0,164.0,17540,671.0,89.0,180.0,26.0,42361.80,1650.0,...,11896.0,9.96,1410.6,2855899.0,201.41,15.28,18681.1,106.0,1123004.0,15175.73
3,73.0,186.0,20.0,6931,262.0,31.0,74.0,6.0,41227.90,721.8,...,-885.0,8.83,530.5,1151042.0,402.10,12.29,18625.2,98.7,712351.0,9626.36
4,58.0,104.0,12.0,4851,350.0,27.0,67.0,13.0,37321.88,580.4,...,1410.0,6.56,456.2,921127.0,185.09,9.70,17259.5,82.0,44377.0,599.69
5,39.0,110.0,12.0,2604,94.0,21.0,21.0,12.0,52172.70,318.2,...,705.0,11.22,260.7,485621.0,143.96,15.82,21532.6,89.4,403023.0,5446.26
6,29.0,63.0,7.0,2577,153.0,16.0,16.0,10.0,38307.22,519.0,...,-969.0,7.28,333.7,724686.0,311.61,5.70,15755.2,107.1,48430.0,654.46
7,58.0,101.0,21.0,5852,421.0,25.0,66.0,17.0,48604.38,456.2,...,1323.0,8.28,275.5,609071.0,145.57,6.43,19293.9,108.0,142046.0,1919.54
8,49.0,111.0,26.0,4498,263.0,38.0,60.0,16.0,37751.68,615.0,...,2680.0,7.33,502.3,987032.0,240.02,9.49,17277.4,100.3,13287.0,179.55
9,40.0,125.0,22.0,3736,182.0,17.0,64.0,5.0,39021.40,300.4,...,-432.0,9.85,238.3,532036.0,207.97,1.77,18009.7,105.2,46958.0,634.57


In [4]:
data_train, data_test = train_test_split(regions_data, test_size=0.1, random_state=42)

## Классификация (но использую регрессоры)

In [5]:
X_train, Y_train = data_train.drop(['Y1','Y2'], axis=1), data_train['Y1']
X_test, Y_test = data_test.drop(['Y1','Y2'], axis=1), data_test['Y1']

model_0 = KNeighborsRegressor()
model_0.fit(X_train, Y_train)

model_1 = DecisionTreeRegressor()
model_1.fit(X_train, Y_train)

model_2 = RandomForestRegressor(n_estimators=1000)
model_2.fit(X_train, Y_train)

model_3 = ExtraTreesRegressor(n_estimators=1000)
model_3.fit(X_train, Y_train)

model_0_test_predictions = [0 if i < 0.05 else 1 for i in model_0.predict(X_test)]
model_1_test_predictions = [0 if i < 0.05 else 1 for i in model_1.predict(X_test)]
model_2_test_predictions = [0 if i < 0.05 else 1 for i in model_2.predict(X_test)]
model_3_test_predictions = [0 if i < 0.05 else 1 for i in model_3.predict(X_test)]

In [6]:
print(
    f'f1_score модели {model_0} составляет {f1_score(Y_test, model_0_test_predictions)}\n'
    f'f1_score модели {model_1} составляет {f1_score(Y_test, model_1_test_predictions)}\n'
    f'f1_score модели {model_2} составляет {f1_score(Y_test, model_2_test_predictions)}\n'
    f'f1_score модели {model_3} составляет {f1_score(Y_test, model_3_test_predictions)}\n'
)

f1_score модели KNeighborsRegressor() составляет 0.0
f1_score модели DecisionTreeRegressor() составляет 0.0
f1_score модели RandomForestRegressor(n_estimators=1000) составляет 0.0
f1_score модели ExtraTreesRegressor(n_estimators=1000) составляет 0.5



In [7]:
model_3.predict(regions_final_test_data)

array([0.006, 0.401, 0.804, 0.255, 0.007, 0.016, 0.   , 0.02 , 0.   ,
       0.   , 0.   , 0.441, 0.11 , 0.   , 0.03 , 0.   , 0.   ])

## Регрессия

In [8]:
X_train, Y_train = data_train.drop(['Y1','Y2'], axis=1), data_train['Y2']
X_test, Y_test = data_test.drop(['Y1','Y2'], axis=1), data_test['Y2']

model_0 = KNeighborsRegressor()
model_0.fit(X_train, Y_train)

model_1 = DecisionTreeRegressor()
model_1.fit(X_train, Y_train)

model_2 = RandomForestRegressor(n_estimators=1000)
model_2.fit(X_train, Y_train)

model_3 = ExtraTreesRegressor(n_estimators=10000)
model_3.fit(X_train, Y_train)

ExtraTreesRegressor(n_estimators=10000)

In [9]:
print(
    f'r2_score модели {model_0} составляет {r2_score(Y_test, model_0.predict(X_test))}\n'
    f'r2_score модели {model_1} составляет {r2_score(Y_test, model_1.predict(X_test))}\n'
    f'r2_score модели {model_2} составляет {r2_score(Y_test, model_2.predict(X_test))}\n'
    f'r2_score модели {model_3} составляет {r2_score(Y_test, model_3.predict(X_test))}\n'
)

r2_score модели KNeighborsRegressor() составляет 0.40723667520346096
r2_score модели DecisionTreeRegressor() составляет 0.5549960406551494
r2_score модели RandomForestRegressor(n_estimators=1000) составляет 0.9209462098875588
r2_score модели ExtraTreesRegressor(n_estimators=10000) составляет 0.9556497648159903



In [10]:
print(model_3.predict(regions_final_test_data))

[ 737.916791  569.932148  772.418173  661.357268  519.830626 1011.007923
  493.667939  776.567275  505.199338  522.542907  470.284632  585.031355
  546.202526  522.783882  563.399397  496.86415   497.560869]
